# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import pickle
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download('punkt')  
nltk.download('stopwords')
import pandas as pd
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
import re
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import  f1_score,  accuracy_score, classification_report, fbeta_score, make_scorer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM messages", engine)
col=[i for i in df.columns if i not in ['id','original', 'genre']]
X = df["message"]
Y = df.iloc[:,4:]
#global category_names
category_names = Y.columns
#print(category_names)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
 pipeline = Pipeline([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        

        ('clf', RandomForestClassifier())
    ])

   



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model =pipeline
model.fit(X_train, y_train) 
model.get_params().keys()

dict_keys(['memory', 'steps', 'text_pipeline', 'clf', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__vect', 'text_pipeline__tfidf', 'text_pipeline__vect__analyzer', 'text_pipeline__vect__binary', 'text_pipeline__vect__decode_error', 'text_pipeline__vect__dtype', 'text_pipeline__vect__encoding', 'text_pipeline__vect__input', 'text_pipeline__vect__lowercase', 'text_pipeline__vect__max_df', 'text_pipeline__vect__max_features', 'text_pipeline__vect__min_df', 'text_pipeline__vect__ngram_range', 'text_pipeline__vect__preprocessor', 'text_pipeline__vect__stop_words', 'text_pipeline__vect__strip_accents', 'text_pipeline__vect__token_pattern', 'text_pipeline__vect__tokenizer', 'text_pipeline__vect__vocabulary', 'text_pipeline__tfidf__norm', 'text_pipeline__tfidf__smooth_idf', 'text_pipeline__tfidf__sublinear_tf', 'text_pipeline__tfidf__use_idf', 'clf__bootstrap', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__min_impu

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = model.predict(X_test)
#print(y_pred)

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    print(labels)
    #accuracy = (y_pred == y_test).mean()    
    #print("f1 score", my_)
   # print("Accuracy:", accuracy)
    #classification_report(y_test, y_pred, target_names=category_names)
    print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))
display_results(y_test, y_pred)

[ 0.  1.]
                        precision    recall  f1-score   support

               related       0.83      0.90      0.87      4996
               request       0.85      0.38      0.52      1104
                 offer       0.00      0.00      0.00        29
           aid_related       0.78      0.41      0.54      2726
          medical_help       0.71      0.02      0.04       535
      medical_products       1.00      0.01      0.03       351
     search_and_rescue       1.00      0.01      0.02       194
              security       0.00      0.00      0.00       137
              military       0.67      0.01      0.02       251
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.14      0.24       408
                  food       0.89      0.25      0.39       730
               shelter       0.89      0.10      0.17       563
              clothing       1.00      0.04      0.07       104
                 money       

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
def build_model():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])

    parameters = {'vect__min_df': [1, 5],
             # 'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[50, 100], 
              #'clf__estimator__min_samples_split':[5],
              #'vect__max_features': (5000, 10000)
       
                 }

    #cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3)   
    #my_scorer = make_scorer(f1_score(y_test, y_pred, average='macro'), greater_is_better=True)
    cv = GridSearchCV(pipeline, param_grid=parameters, scoring="f1_weighted")


    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
model=build_model()
model.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [1, 5], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [9]:
y_pred = model.predict(X_test)
display_results(y_test, y_pred)

[0 1]
                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4996
               request       0.91      0.47      0.62      1104
                 offer       0.00      0.00      0.00        29
           aid_related       0.78      0.62      0.69      2726
          medical_help       0.57      0.08      0.14       535
      medical_products       0.83      0.10      0.18       351
     search_and_rescue       0.88      0.08      0.14       194
              security       0.50      0.01      0.01       137
              military       0.82      0.06      0.10       251
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.32      0.48       408
                  food       0.81      0.63      0.71       730
               shelter       0.84      0.39      0.54       563
              clothing       0.73      0.08      0.14       104
                 money       0.80

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
def build_model_new():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(ExtraTreesClassifier()))])

    parameters = {'vect__min_df': [1, 5],
             # 'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[50, 100], 
             
                 }

    #cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3)   
    #my_scorer = make_scorer(f1_score(y_test, y_pred, average='macro'), greater_is_better=True)
    cv = GridSearchCV(pipeline, param_grid=parameters, scoring="f1_weighted")
    return cv
model=build_model_new()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
display_results(y_test, y_pred)

[0 1]
                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4996
               request       0.91      0.47      0.62      1104
                 offer       0.00      0.00      0.00        29
           aid_related       0.79      0.65      0.72      2726
          medical_help       0.61      0.08      0.14       535
      medical_products       0.90      0.10      0.18       351
     search_and_rescue       0.83      0.03      0.05       194
              security       0.33      0.01      0.01       137
              military       0.71      0.05      0.09       251
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.24      0.39       408
                  food       0.85      0.43      0.57       730
               shelter       0.85      0.36      0.50       563
              clothing       0.74      0.13      0.23       104
                 money       0.88

I obtained best result with Random Forest classifier with mean f1 score 0.57. I used also Adaboost it was very slow that is why I removed it, when I needed to rerun notebook and result was not that good.

### 9. Export your model as a pickle file

In [12]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.